In [4]:
import polars as pl
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import json
from io import StringIO
import pygwalker as pyg
import os
from tqdm import tqdm

reading competition editions json

In [5]:
ce_json_path = './skillcorner-competition-editions.json'
ce_df1 = pl.read_json(ce_json_path)
ce_df1.head(1)
#ce_df.columns

count,next,previous,results
i64,null,null,list[struct[4]]
938,null,null,"[{0,{0,""FIFA"",""World Cup"",""male"",""adult""},{10,2014,2014,""2014""},""FIFA - World Cup - 2014""}, {1,{1,""ENG"",""Premier League"",""male"",""adult""},{2,2015,2016,""2015/2016""},""ENG - Premier League - 2015/2016""}, … {1129,{221,""INT"",""Pinatar National Team Friendlies"",""male"",""U21""},{128,2025,2025,""2025""},""INT - Pinatar National Team Friendlies - 2025""}]"


In [6]:
#ce_df2 = ce_df1.explode('results')
ce_df2 = ce_df1.select('results').explode('results')
ce_df2 = ce_df2.unnest('results')
ce_df2.head(1)

id,competition,season,name
i64,struct[5],struct[4],str
0,"{0,""FIFA"",""World Cup"",""male"",""adult""}","{10,2014,2014,""2014""}","""FIFA - World Cup - 2014"""


In [7]:
ce_name_id = ce_df2.select([
    pl.col('name'),
    pl.col('id'),
    ]
)
ce_name_id.head(1)

name,id
str,i64
"""FIFA - World Cup - 2014""",0


In [8]:
ce_competition = ce_df2.select('competition').unnest('competition')
ce_competition = ce_competition.rename({col: f"competition_{col}" for col in ce_competition.columns})
ce_competition.head(1)

competition_id,competition_area,competition_name,competition_gender,competition_age_group
i64,str,str,str,str
0,"""FIFA""","""World Cup""","""male""","""adult"""


In [9]:
ce_season = ce_df2.select('season').unnest('season')
ce_season = ce_season.rename({col: f"season_{col}" for col in ce_season.columns})
ce_season.head(1)

season_id,season_start_year,season_end_year,season_name
i64,i64,i64,str
10,2014,2014,"""2014"""


In [10]:
ce = pl.concat([
    ce_name_id,
    ce_season,
    ce_competition
],
how = "horizontal",)
ce.head(10)

name,id,season_id,season_start_year,season_end_year,season_name,competition_id,competition_area,competition_name,competition_gender,competition_age_group
str,i64,i64,i64,i64,str,i64,str,str,str,str
"""FIFA - World Cup - 2014""",0,10,2014,2014,"""2014""",0,"""FIFA""","""World Cup""","""male""","""adult"""
"""ENG - Premier League - 2015/20…",1,2,2015,2016,"""2015/2016""",1,"""ENG""","""Premier League""","""male""","""adult"""
"""UEFA - European Championship -…",2,12,2016,2016,"""2016""",2,"""UEFA""","""European Championship""","""male""","""adult"""
"""ENG - Premier League - 2016/20…",3,3,2016,2017,"""2016/2017""",1,"""ENG""","""Premier League""","""male""","""adult"""
"""ENG - Premier League - 2017/20…",4,4,2017,2018,"""2017/2018""",1,"""ENG""","""Premier League""","""male""","""adult"""
"""FRA - Ligue 1 - 2016/2017""",5,3,2016,2017,"""2016/2017""",3,"""FRA""","""Ligue 1""","""male""","""adult"""
"""FRA - Ligue 1 - 2017/2018""",6,4,2017,2018,"""2017/2018""",3,"""FRA""","""Ligue 1""","""male""","""adult"""
"""ESP - LaLiga - 2016/2017""",7,3,2016,2017,"""2016/2017""",4,"""ESP""","""LaLiga""","""male""","""adult"""
"""ESP - LaLiga - 2017/2018""",8,4,2017,2018,"""2017/2018""",4,"""ESP""","""LaLiga""","""male""","""adult"""


reading json player file- test on 1 file


In [11]:
position_json_testpath = './skillcorner-20250214/metrics-ce249-rAM.json'

position_test_df = pl.read_json(position_json_testpath)

In [12]:
position_test_df.columns
#position_test_df.head(10)

['Count High Acceleration OTIP 1',
 'Minutes TIP 1',
 'short_name',
 'Count High Deceleration 2',
 'player_birthdate',
 'Count Medium Deceleration 2',
 'Count Medium Deceleration OTIP 1',
 'Count High Deceleration TIP 1',
 'Running Distance OTIP 1',
 'Count Medium Deceleration OTIP',
 'HSR Distance OTIP 1',
 'Count Sprint 1',
 'Distance 2',
 'Count High Deceleration TIP',
 'Count Medium Acceleration OTIP 2',
 'Count Medium Acceleration',
 'competition_id',
 'Running Distance OTIP 2',
 'Count HSR OTIP',
 'Count Sprint TIP',
 'Sprinting Distance 1',
 'season_name',
 'Distance 1',
 'competition_edition_id',
 'Count High Acceleration 1',
 'Count High Deceleration',
 'Count HSR 2',
 'Distance TIP 2',
 'HSR Distance 2',
 'Count Sprint OTIP',
 'Sprinting Distance TIP 2',
 'Count HSR TIP',
 'HSR Distance TIP 2',
 'Count Medium Acceleration TIP 2',
 'Minutes OTIP 2',
 'Count Medium Deceleration 1',
 'season_id',
 'Sprinting Distance',
 'Count Medium Deceleration',
 'Count High Acceleration OTIP

In [13]:
#define the path to the entire directory
json_dir_path = './skillcorner-20250214/'

### Look at correlations between parameters
Looking at the correlations across all competitions across all positions to see if we can leave out certain parameters

In [14]:
json_list = os.listdir(json_dir_path)
#json_list

In [15]:
read_json_list = []

for jsonf in tqdm(json_list):
    json_path = f"{json_dir_path}{jsonf}"
    json_df = pl.read_json(json_path)

    read_json_list.append(json_df)

corr_df = pl.concat(read_json_list)

100%|██████████| 1652/1652 [00:49<00:00, 33.43it/s]


In [16]:
json_df.shape

(449537, 117)

In [ ]:
#create the correlation matrix

In [ ]:
corr_df.to_numpy()

In [ ]:
pyg.walk(position_test_df)